In [1]:
%%bash
nrnivmodl

/usr/bin/xcrun
/Users/anatolii-sorokin/Documents/Projects/neuro-trafic
Mod files: "./h.mod" "./kadist.mod" "./kaprox.mod" "./kdrca1.mod" "./km.mod" "./na3n.mod" "./naxn.mod"

 -> Compiling mod_func.cpp
 => LINKING shared library ./libnrnmech.dylib
 => LINKING executable ./special LDFLAGS are:    
Successfully created arm64/special


In [2]:
# Download NEURON: http://www.neuron.yale.edu/neuron/download
# Download PyNeuronToolbox: https://github.com/ahwillia/PyNeuron-Toolbox

from neuron import h
import numpy as np
import math
import pylab as plt
import scipy.linalg
from PyNeuronToolbox.record import ez_record,ez_convert
from PyNeuronToolbox.morphology import shapeplot,shapeplot_animate
from mpl_toolkits.mplot3d import Axes3D
from JSAnimation import IPython_display
from matplotlib import animation
from matplotlib.pyplot import cm
%matplotlib inline
np.random.seed(123456789)

DC = 10 # diffusion coeff [microns^2 s^-1]

# Load morphology and other stuff
# --> SegLists: soma[2], dend[74], dend_5[37], apic[42], axon[1]
# --> Files from Migliore & Migliore (2012)
# --> CA1 pyramidal neuron
h.load_file('stdrun.hoc')
h.xopen('ri06.hoc')
h.xopen('fixnseg.hoc')
h.xopen('5a_nogui.hoc')
h.tstop = 700.0

	0 
total # of segments (50Hz): 742   max path distance: 853.32493 
apical 0  dend_5[0]
apical 1  dend_5[15]
apical 2  dend_5[14]
apical 3  dend_5[13]
apical 4  dend_5[12]
apical 5  dend_5[12]
apical 6  dend_5[12]
apical 7  dend_5[12]
apical 8  dend_5[12]
apical 9  dend_5[12]
apical 10  dend_5[12]
apical 11  dend_5[11]
apical 12  dend_5[11]
apical 13  dend_5[11]
apical 14  dend_5[10]
apical 15  dend_5[9]
apical 16  dend_5[8]
apical 17  dend_5[8]
apical 18  dend_5[8]
apical 19  dend_5[7]
apical 20  dend_5[6]
apical 21  dend_5[5]
apical 22  dend_5[5]
apical 23  dend_5[5]
apical 24  dend_5[5]
apical 25  dend_5[5]
apical 26  dend_5[5]
apical 27  dend_5[5]
apical 28  dend_5[4]
apical 29  dend_5[4]
apical 30  dend_5[4]
apical 31  dend_5[3]
apical 32  dend_5[2]
apical 33  dend_5[2]
apical 34  dend_5[2]
apical 35  dend_5[2]
apical 36  dend_5[2]
apical 37  dend_5[2]
apical 38  dend_5[2]
apical 39  dend_5[1]
apical 40  dend_5[1]
apical 41  dend_5[1]
#1


In [3]:
from PyNeuronToolbox.morphology import dist_between,allsec_preorder

def dist_to_soma(segment):
    return dist_between(h,h.soma[0](0.5),segment)

#seglist in pre-order
sec_list = allsec_preorder(h)
seg_list = []
for sec in sec_list:
    locs = np.linspace(0,1,sec.nseg+2)[1:-1]
    for loc in locs:
        seg_list.append(sec(loc))
n = len(seg_list)
N = n
day7 = 7*24*3600


In [30]:
len(seg_list)

742

In [4]:
for i in range(len(sec_list)):
    sec = sec_list[i]
    print(f"{i}: {sec.name()}, nseg={sec.nseg}, L={sec.L}, or={sec.orientation()}, hname={sec.hname()}")

0: soma[0], nseg=1, L=15.166100084781647, or=0.0, hname=soma[0]
1: soma[1], nseg=1, L=10.883900105953217, or=0.0, hname=soma[1]
2: dend_5[0], nseg=1, L=27.29313676464808, or=0.0, hname=dend_5[0]
3: apic[0], nseg=9, L=147.93433051396065, or=0.0, hname=apic[0]
4: dend_5[1], nseg=3, L=57.45149110486393, or=0.0, hname=dend_5[1]
5: apic[39], nseg=1, L=23.29171407802628, or=0.0, hname=apic[39]
6: apic[41], nseg=3, L=34.922839435595144, or=0.0, hname=apic[41]
7: apic[40], nseg=5, L=52.39168693889804, or=0.0, hname=apic[40]
8: dend_5[2], nseg=1, L=21.183727600433784, or=0.0, hname=dend_5[2]
9: apic[32], nseg=1, L=12.617619533060896, or=0.0, hname=apic[32]
10: apic[38], nseg=13, L=208.40219120020532, or=0.0, hname=apic[38]
11: apic[33], nseg=1, L=9.679981164937955, or=0.0, hname=apic[33]
12: apic[37], nseg=3, L=32.69642802210867, or=0.0, hname=apic[37]
13: apic[34], nseg=1, L=6.683028121429655, or=0.0, hname=apic[34]
14: apic[36], nseg=5, L=58.600785345533666, or=0.0, hname=apic[36]
15: apic[35

In [27]:
(len(locs),len(sec_list),len(seg_list),[sec.nseg for sec in sec_list],np.sum([sec.nseg for sec in sec_list]))

(5,
 156,
 742,
 [1,
  1,
  1,
  9,
  3,
  1,
  3,
  5,
  1,
  1,
  13,
  1,
  3,
  1,
  5,
  7,
  1,
  3,
  1,
  3,
  9,
  5,
  1,
  3,
  11,
  1,
  7,
  3,
  7,
  5,
  1,
  5,
  1,
  3,
  1,
  1,
  11,
  15,
  1,
  11,
  1,
  13,
  1,
  5,
  7,
  7,
  1,
  3,
  11,
  1,
  3,
  3,
  7,
  5,
  3,
  11,
  1,
  11,
  3,
  3,
  1,
  5,
  15,
  5,
  15,
  5,
  7,
  9,
  1,
  3,
  5,
  3,
  11,
  3,
  9,
  7,
  1,
  3,
  5,
  7,
  7,
  1,
  1,
  1,
  9,
  11,
  1,
  9,
  7,
  1,
  7,
  1,
  9,
  1,
  7,
  7,
  1,
  1,
  1,
  1,
  7,
  11,
  1,
  1,
  9,
  3,
  1,
  7,
  7,
  9,
  3,
  3,
  5,
  5,
  7,
  3,
  5,
  7,
  7,
  1,
  9,
  3,
  5,
  3,
  5,
  5,
  3,
  3,
  3,
  3,
  7,
  1,
  7,
  3,
  3,
  3,
  1,
  5,
  9,
  9,
  1,
  13,
  1,
  1,
  3,
  9,
  7,
  7,
  9,
  5,
  3,
  3,
  5,
  5,
  3,
  5],
 742)

In [26]:
import re
ex_re=re.compile('^(dend|apic).*')
(len([i for i in sec_list if ex_re.search(i.name()) is not None]),len([i for i in seg_list if ex_re.search(str(i)) is not None]),len([i for i in seg_list if ex_re.search(str(i)) is not None])/len([i for i in sec_list if ex_re.search(i.name()) is not None]))

(153, 735, 4.803921568627451)

In [21]:
ex_re.search(str(seg_list[0]))
#(str((seg_list[:5])))

In [7]:
(dist_to_soma(sec(0)),dist_to_soma(sec(1)),sec.L,np.linspace(0,1,sec.nseg+2),np.linspace(0,1,sec.nseg+2)[1:-1])

(7.583050042390823,
 104.674267724172,
 97.09121768178117,
 array([0.        , 0.16666667, 0.33333333, 0.5       , 0.66666667,
        0.83333333, 1.        ]),
 array([0.16666667, 0.33333333, 0.5       , 0.66666667, 0.83333333]))

In [ ]:
dts = [dist_to_soma(s) for s in seg_list]
dts[:11]

In [24]:
import re
import pandas as pd
p = re.compile("^([^\\[]+)\\[.+")
tps = [p.sub("\\1",seg.sec.name()) for seg in seg_list]
pd.Series(tps).drop_duplicates().tolist()

['soma', 'dend_5', 'apic', 'dend', 'axon']

In [ ]:
def sushi_system(a,b,c,d,l):
    """
    Returns a matrix A, such that dx/dt = A*x
    
    N = # of compartments
    A is (2N x 2N) matrix
    x is (2N x 1) vector.
      The first N elements correspond to concentrations of u (molecules in transit)
      The second half correspond to concentrations of u-star (active molecules)
    The trafficking rate constants along the microtubules are given by the vectors "a" and "b"
    The rate constants for u turning into u* is given by the vector "c"
    The rate constants for u* turning into u is given by the vector "d"
    The rate constants for the degradation of u* is given by the vector "l"
    """
    # number of compartments
    N = len(l)
    
    ## State-space equations
    #  dx/dt = Ax + Bu
    A = np.zeros((2*N,2*N))

    # Trafficking along belt
    # Iterative traversal of dendritic tree in pre-order
    i = 0
    section = None
    parentStack = [(None,h.soma[0])]
    while len(parentStack)>0:
        # Get next section to traverse
        #  --> p is parent index, section is h.Section object
        (p,section) = parentStack.pop()
        
        # Trafficking to/from parent
        if p is not None:
            # Out of parent, into child
            ai = a.pop()
            A[p,p] += -ai
            A[i,p] += ai
            # Into parent, out of child
            bi = b.pop()
            A[p,i] += bi
            A[i,i] += -bi
        
        # visit all segments in compartment
        for (j,seg) in enumerate(section):
            # Deal with out/into rates within compartment, just tridiag matrix
            if j>0:
                # Out of parent, into child
                ai = a.pop()
                A[i-1,i-1] += -ai
                A[i,i-1] += ai
                # Into parent, out of child
                bi = b.pop()
                A[i-1,i] += bi
                A[i,i] += -bi
            # move onto next compartment
            i += 1
        
        # now visit children in pre-order
        child_list = list(h.SectionRef(sec=section).child)
        if child_list is not None:
            child_list.reverse()
        for c_sec in child_list:
            parentStack.append([i-1,c_sec]) # append parent index and child
    
    # Trafficking off the belt
    for i in range(N):
        A[i,i] += -c[i]
        A[i+N,i] += c[i]
    
    # Reattachment to belt
    #for i in range(N):
    #    # reattachment
    #    A[i, i + N] += d[i]
    #    A[i + N, i + N] += -d[i]

    # Degradation after being taken off the belt
    for i in range(N):
        A[i+N,i+N] = -l[i]
    
    return A

In [ ]:
def trafficking_solution(utarg):
    """ Solve the problem by tuning trafficking rates, like Figs 1 and 2. """
    x = []
    
    # Iterative traversal of dendritic tree in pre-order
    i = 0
    section = None
    parentStack = [(None,h.soma[0])]
    while len(parentStack)>0:
        # Get next section to traverse
        #  --> p is parent index, section is h.Section object
        (p,section) = parentStack.pop()
        
        # Trafficking to/from parent
        if p is not None:
            mp = utarg[p] # concentration in parent
            mc = utarg[i] # concentration in child
            x.insert(0,mp/mc)
        
        # visit all segments in compartment
        for (j,seg) in enumerate(section):
            # Deal with out/into rates within compartment, just tridiag matrix
            if j>0:
                mp = utarg[i-1]
                mc = utarg[i]
                x.insert(0,mp/mc)
                
            # move onto next compartment
            i += 1
        
        # now visit children in pre-order
        child_list = list(h.SectionRef(sec=section).child)
        if child_list is not None:
            child_list.reverse()
        for c_sec in child_list:
            parentStack.append([i-1,c_sec]) # append parent index and child
    
    # return calculated guesses (flip, up/down since get_deriv pops from start)
    return np.array(x)

def get_sys_matrix(utarg,F=0.5,Ctau=1e-3,dscale=0.1,dv=1e-7):
    # F is a mixing factor between 0 and 1
    K = np.sum(utarg)/n
    x = trafficking_solution(F*utarg + (1-F)*K)
    a = (1/(1+x))
    a = list(a)
    b = list((1/(1+x**-1)))
    l = list(np.ones(n)*dv)
    c = list(Ctau*utarg/(F*utarg + (1-F)*K))
    d = list([ci * dscale for ci in c])
    A = sushi_system(a,b,c,d,l)
    return A

In [ ]:
# matrix exponential used to solve linear system
from scipy.linalg import expm

# initial condition, u starts only in soma and is trafficked out to dendrites
u0 = np.zeros(2*n)
u0[0] = 1.0  # compartment 0 is the soma

In [ ]:
def solve_u(u0,w,V,Vinv,t):
    D = np.diag(np.exp(w*t))          # diagonal matrix exponential
    PHI = np.real(V.dot(D.dot(Vinv))) # state transition matrix
    return PHI.dot(u0)                # calculate u(t)

def sim_time(A,u0,time,nframes=10):
    # Run a simulation (log time)
    # --> this is a linear system; thus, matrix exponential provides exact solution
    utrace = [u0]
    w,V = scipy.linalg.eig(A)
    Vinv = np.linalg.inv(V)
    t = np.logspace(-0.5,math.log10(time),nframes)
    for t_ in t: utrace.append(solve_u(u0,w,V,Vinv,t_))
    return np.array(utrace).T

def run_sim(A,nframes=10):
    # Run a simulation (log time)
    # --> this is a linear system; thus, matrix exponential provides exact solution
    utrace = [u0]
    t = np.logspace(-0.5,8.5,nframes)
    w,V = scipy.linalg.eig(A)
    Vinv = np.linalg.inv(V)
    t = np.logspace(-0.5,8.5,nframes)
    for t_ in t: utrace.append(solve_u(w,V,Vinv,t_))
    return np.array(utrace).T

def animate_sim(climits=[0,0.0035]):
    u = utrace[n:,:]
    fig = plt.figure(figsize=(8,8))
    shapeax = plt.subplot(111, projection='3d')
    lines = shapeplot(h,shapeax,order='pre',lw=2)
    shapeax.view_init(elev=80,azim=-90)
    plt.title('distribution of active/detached cargo',fontweight='bold',fontsize=14)
    plt.axis('off')

    # Hack to get a colorbar
    # http://stackoverflow.com/questions/8342549/matplotlib-add-colorbar-to-a-sequence-of-line-plots
    sm = plt.cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(climits[0],climits[1])) 
    #sm._A = []
    #plt.colorbar(sm, shrink=0.5)
    #plt.colorbar(sm)
    plt.tight_layout()

    anim_func = shapeplot_animate(u.T,lines,u.shape[1],clim=climits,cmap=cm.cool)
    return animation.FuncAnimation(fig, anim_func, frames=u.shape[1], interval=5e2, blit=True)

def snap_ss(name,clim=[0,0.0035],cmap=plt.cm.cool):
    u = utrace[n:,:]
    fig = plt.figure(figsize=(8,8))
    shapeax = plt.subplot(111, projection='3d')
    lines = shapeplot(h,shapeax,cvals=u[:,-1],clim=clim,\
                      cmap=cmap,order='pre',lw=2)
    shapeax.view_init(elev=80,azim=-90)
    plt.title('steady-state',fontweight='bold',fontsize=14)

    # colorbar
    sm = cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(clim[0],clim[1])) 
    #sm = plt.cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm._A = []
    #plt.colorbar(sm, shrink=0.5)
    #plt.colorbar(sm,cax=shapeax)
    #plt.colorbar()
    plt.tight_layout()
    plt.axis('off')
    #plt.savefig(name+'.svg')

Read Experimental Data
----

In [25]:
tdf=pd.read_csv('data/seg_mapping.csv')
abbCA1=tdf['abb']
abbT={}
segIdx={}
for i in range(n):
    abbT[abbCA1[i]] = 1+ abbT.get(abbCA1[i],0)
    ll=segIdx.get(abbCA1[i],[])
    ll.append(i)
    segIdx[abbCA1[i]] = ll

(abbCA1,abbT,segIdx.keys(),{k:len(segIdx.get(k)) for k in segIdx.keys()})

(0       Soma_0
 1       Soma_0
 2      CA1sr_1
 3      CA1sr_1
 4      CA1sr_2
         ...   
 737        NaN
 738        NaN
 739        NaN
 740        NaN
 741        NaN
 Name: abb, Length: 742, dtype: object,
 {'Soma_0': 2,
  'CA1sr_1': 3,
  'CA1sr_2': 6,
  'CA1sr_3': 29,
  'CA1sr_4': 37,
  'CA1sr_5': 48,
  'CA1sr_6': 44,
  'CA1sr_7': 46,
  'CA1sr_8': 38,
  'CA1sr_9': 21,
  'CA1sr_10': 9,
  'CA1slm_1': 13,
  'CA1slm_2': 24,
  'CA1slm_3': 35,
  'CA1slm_4': 29,
  'CA1slm_5': 9,
  'CA1so_5': 16,
  'CA1so_4': 100,
  'CA1so_3': 134,
  'CA1so_2': 85,
  'CA1so_1': 9,
  nan: 5},
 dict_keys(['Soma_0', 'CA1sr_1', 'CA1sr_2', 'CA1sr_3', 'CA1sr_4', 'CA1sr_5', 'CA1sr_6', 'CA1sr_7', 'CA1sr_8', 'CA1sr_9', 'CA1sr_10', 'CA1slm_1', 'CA1slm_2', 'CA1slm_3', 'CA1slm_4', 'CA1slm_5', 'CA1so_5', 'CA1so_4', 'CA1so_3', 'CA1so_2', 'CA1so_1', nan]),
 {'Soma_0': 2,
  'CA1sr_1': 3,
  'CA1sr_2': 6,
  'CA1sr_3': 29,
  'CA1sr_4': 37,
  'CA1sr_5': 48,
  'CA1sr_6': 44,
  'CA1sr_7': 46,
  'CA1sr_8': 38,
  'CA1sr_9'

In [ ]:
expD=pd.read_csv('data/CA1_gradient.csv')
subreg = ['CA1so', 'CA1sr', 'CA1slm']
expD

In [ ]:
import sushibelt
import time

In [ ]:
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= i
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"segment_distribution",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= [j for j in range(len(subreg)) if subreg[j]== expD['Subregion'][i]][0]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"subregion_distribution",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

Day 0 3 weeks
------

In [ ]:
cname='D0W3'
utarg = np.ones(len(seg_list))
unsig = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]/len(sidx)
    unsig[sidx] *= len(sidx)
    v=expD[f"{cname}_MEAN"][i]
    print(f"i={i}, abb={abb}, len(sidx)={len(sidx)}, v={v}, nv={v/len(sidx)}")
u1 = np.concatenate((utarg,utarg))
un = np.concatenate((unsig,unsig))
utrace = [u1,u1]
utrace = np.array(utrace).T
#snap_ss(f"case1_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
utM,utF = sushibelt.aggregate_segments(u1/np.sum(u1),segIdx,expD['Abbreviation'],fun=np.sum)
x=np.array(range(expD.shape[0]))

plt.plot(x,utM,label='utargM')
plt.plot(x,utF,label='utargF')
# show a legend on the plot
plt.legend()


In [ ]:
utM,utF = sushibelt.aggregate_segments(un/np.sum(un),segIdx,expD['Abbreviation'],fun=np.sum)
x=np.array(range(expD.shape[0]))

plt.plot(x,utM,label='utargM')
plt.plot(x,utF,label='utargF')
# show a legend on the plot
plt.legend()


In [ ]:
cname='D7W3'
obsD = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    obsD[sidx] *= expD[f"{cname}_MEAN"][i]/len(sidx)
nobsD = np.array(expD[f"{cname}_MEAN"])/np.sum(expD[f"{cname}_MEAN"])
uD = np.concatenate((obsD,obsD))
utrace = [uD,uD]
utrace = np.array(utrace).T
snap_ss(f"case1_{cname}_target",clim=[0,max(obsD)],cmap=plt.cm.YlOrBr)
max(obsD)

In [ ]:
diF=abs(uD-u1)
utrace = [diF,diF]
utrace = np.array(utrace).T
snap_ss(f"case1_{cname}_target",clim=[min(diF),max(diF)],cmap=plt.cm.YlOrBr)
(min(diF),max(diF))

In [ ]:
x=np.array(range(expD.shape[0]))
plt.plot(x,expD[f"D0W3_MEAN"],label='D0W3')
plt.plot(x,expD[f"D7W3_MEAN"],label='D7W3')
# show a legend on the plot
plt.legend()
 
# function to show the plot
plt.show()

In [ ]:
plt.plot(x,expD[f"D0W3_MEAN"]/np.sum(expD[f"D0W3_MEAN"]),label='norm D0W3')
plt.plot(x,expD[f"D7W3_MEAN"]/np.sum(expD[f"D7W3_MEAN"]),label='norm D7W3')
# show a legend on the plot
plt.legend()
 
# function to show the plot
plt.show()

In [ ]:
mProp=0.25
u0 = np.concatenate((mProp*utarg/np.sum(utarg),(1-mProp)*utarg/np.sum(utarg)))
initM,initF = sushibelt.aggregate_segments(u0,segIdx,expD['Abbreviation'],fun=np.sum)
x=np.array(range(expD.shape[0]))

plt.plot(x,expD[f"D0W3_MEAN"]/np.sum(expD[f"D0W3_MEAN"]),label='D0W3_MEAN')
plt.plot(x,initM,label='mobile')
plt.plot(x,initF,label='detached')
# show a legend on the plot
plt.legend()
 
# function to show the plot
plt.show()

In [ ]:
mProp=0.25
u0 = np.concatenate((mProp*utarg/np.sum(utarg),(1-mProp)*utarg/np.sum(utarg)))
initM,initF = sushibelt.aggregate_segments(u0,segIdx,expD['Abbreviation'],fun=np.sum)
utM,utF = sushibelt.aggregate_segments(uD/np.sum(u1),segIdx,expD['Abbreviation'],fun=np.sum)
x=np.array(range(expD.shape[0]))

plt.plot(x,utM,label='utarg')
plt.plot(x,initM,label='mobile')
plt.plot(x,initF,label='detached')
# show a legend on the plot
plt.legend()
 
# function to show the plot
plt.show()

In [ ]:
(mProp,np.sum(utM),np.sum(utF),np.sum(initM),np.sum(initF),np.sum(u1))

First Case: F=0, Ctau=5e-4, dv=1e-7
----

In [ ]:
#utarg = utrace#[,0]
utarg /= np.sum(utarg)
F=0
Ctau=5e-4
dscale=0.1
dv=1e-7
mProp=0.25
K = np.sum(utarg)/n
x = trafficking_solution(F*utarg + (1-F)*K)
a = (1/(1+x))
a = list(a)
b = list((1/(1+x**-1)))
l = list(np.ones(n)*dv)
c = list(Ctau*utarg/(F*utarg + (1-F)*K))
d = list(np.zeros(n))
AF1 = sushi_system(a,b,c,d,l)
a, b, c, _ = sushibelt.set_uniform_rates(h, DC)
A = sushi_system(a,b,c,d,l)

In [ ]:
# Run simulation
#utarg = utrace
#utarg /= np.sum(utarg)
u0 = np.concatenate((mProp*utarg,(1-mProp)*utarg))
initTime=time.time()
#Ai = get_sys_matrix(utarg,F=0,Ctau=1e-5,dscale=1.0,dv=0)
#A = get_sys_matrix(utarg,F=0,Ctau=1e-5,dscale=0.01,dv=0)
utrace = sim_time(A,u0,day7)
FinalTime=time.time()-initTime

# snapshot of steady-state
#snap_ss(f"case1_{cname}",clim=[0,max(utrace[:,-1])],cmap=plt.cm.YlOrBr)

# animation
#animate_sim()
print(FinalTime)

In [ ]:
(np.sum(u0),np.sum(utrace))

In [ ]:
# Run simulation
#utarg = utrace
#utarg /= np.sum(utarg)
u0 = np.concatenate((mProp*utarg,(1-mProp)*utarg))
initTime=time.time()
#Ai = get_sys_matrix(utarg,F=0,Ctau=1e-5,dscale=1.0,dv=0)
#A = get_sys_matrix(utarg,F=0,Ctau=1e-5,dscale=0.01,dv=0)
utraceF1 = sim_time(AF1,u0,day7)
FinalTime=time.time()-initTime

# snapshot of steady-state
snap_ss(f"case1_{cname}",clim=[0,max(utraceF1[:,-1])],cmap=plt.cm.YlOrBr)

# animation
#animate_sim()
print(FinalTime)

In [ ]:
(np.sum(u0),np.sum(utrace),np.sum(utraceF1))

In [ ]:
resM,resF = sushibelt.aggregate_segments(utrace[:,-1],segIdx,expD['Abbreviation'],fun=np.sum)
x=np.array(range(expD.shape[0]))
plt.plot(x,resM,label='mobile')
plt.plot(x,resF,label='detached')
# show a legend on the plot
plt.legend()
 
# function to show the plot
plt.show()

In [ ]:
resMf1,resFf1 = sushibelt.aggregate_segments(utraceF1[:,-1],segIdx,expD['Abbreviation'],fun=np.sum)
x=np.array(range(expD.shape[0]))

plt.plot(x,resM,label='mobile DC')
plt.plot(x,resF,label='detached DC')
plt.plot(x,resMf1,label='mobile F1')
plt.plot(x,resFf1,label='detached F1')
# show a legend on the plot
plt.legend()
 
# function to show the plot
plt.show()

In [ ]:
abb_list=np.array(expD['Abbreviation'])
#(len(abb_list),np.mean(expD[f"{cname}_MEAN"]),abb_list,segIdx)

In [ ]:
cname0='D0M3'
cname7='D7M3'
obsD = np.ones(len(seg_list))
resMo = np.zeros(expD.shape[0])
resFo = np.zeros(expD.shape[0])
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    obsD[sidx] *= expD[f"{cname7}_MEAN"][i]
nobsD = np.array(expD[f"{cname7}_MEAN"])/np.sum(expD[f"{cname7}_MEAN"])
nobsDnn = np.array(expD[f"{cname7}_MEAN"])/np.sum(expD[f"{cname0}_MEAN"])
target = np.array(expD[f"{cname7}_MEAN"])/np.sum(expD[f"{cname0}_MEAN"]) #results to fit to
targSD = np.array(expD[f"{cname7}_SD"])/np.sum(expD[f"{cname0}_MEAN"]) #measurement errors
(np.sum(target),nobsD)

In [ ]:
?ax.legend

In [ ]:
(resF,nobsD,np.sum((resF-nobsD)**2))

In [ ]:
x=np.array(range(expD.shape[0]))

plt.plot(x,nobsDnn,'o-',label=cname7)
#plt.plot(x,nresM,label='mobile O')
#plt.plot(x,nresF,label='detached O')
plt.plot(x,resM,label='mobile DC')
plt.plot(x,resF,label='detached DC')
plt.plot(x,resMf1,label='mobile F1')
plt.plot(x,resFf1,label='detached F1')
# show a legend on the plot
plt.legend()
 
# function to show the plot
plt.show()

In [ ]:
itarg = np.ones(len(seg_list), dtype=int)
par = [1000,0,0,0,1,2,3,4]
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx = segIdx[abb]
    itarg[sidx] *= [j + 4 for j in range(len(subreg)) if subreg[j] == expD['Subregion'][i]][0]
utarg = [par[j] for j in itarg]
(itarg,utarg)

In [ ]:
def calcUtrace(par,delta=1e-7):
    F = par[0]
    Ctau = 10 ** par[1]
    dv = 10 ** par[2]
    mProp = par[3]
    utarg = delta*np.ones(N)
    for k in range(N):
        if itarg[k] > 3:
            utarg[k] = par[itarg[k]]
    utarg /= np.sum(utarg)
    K = np.sum(utarg) / N
    x = trafficking_solution(F * utarg + (1 - F) * K)
    a = (1 / (1 + x))
    a = list(a)
    b = list((1 / (1 + x ** -1)))
    l = list(np.ones(N) * dv)
    c = list(Ctau * utarg / (F * utarg + (1 - F) * K))
    d = list(np.zeros(N))
    A = sushi_system(a, b, c, d, l)
    u0 = np.concatenate((mProp * dinit, (1 - mProp) * dinit))
    utrace = sim_time(A, u0, day7)
    return utrace

d0w = np.ones(n)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d0w[sidx] *= expD[f"{cname0}_MEAN"][i]/len(sidx)
dinit = d0w/np.sum(d0w)
#par=[0.961919484252899,-5.133858267716535,-8.0,0.31251612903225806,0.001,0.4316110587484276,0.9295395757281527]
#par=[9.59958074e-01, -5.13385827e+00, -8.40000000e+00,  3.12516129e-01,
#        1.00095272e-03,  4.47587470e-01,  9.64900400e-01]
#par = [0.7319544570315621,-18.0,0.9999999999999858,0.09013312454954349,0.0010182254872917412,0.558826997851783,0.9999999999999996]
#cost=0.22173789
#par = [0.0,-5.928312450225743,-5.432607526718705,0.04433935470301702,0.001,0.027516497491107825,1.0]
#cost = 0.00011137664851494792
#par = [7.01114319640285e-11,-7.845116636109227,-5.370033238533027,0.9138982830508748,0.001,0.4710826042662421,0.9999999999999994]
#cost = 8.913377924831686e-05
par = [0.0,-7.358806506113698,-5.877206062417285,0.8259346892916518,0.001,0.005597295489466257,0.007967029322102446]
cost = 0.0002469349585610918
mProp = par[3]
utrace = calcUtrace(par)
resM, resF = sushibelt.aggregate_segments(utrace[:, -1], segIdx, expD['Abbreviation'], fun=np.sum)


In [ ]:
{'F' : par[0],
'Ctau' : 10 ** par[1],
'l' : 10 ** par[2],
'mProp' : par[3],
'demand_CA1so': par[4],
'demand_CA1sr': par[5],
'demand_CA1slm': par[6],
'sum_target' : np.sum(target),
'sum_target^2' : np.sum(target ** 2),
'sum_resM' : np.sum(resM),
'sum_resF' : np.sum(resF),
'sum_tot' : np.sum(resF)+np.sum(resM),
'err' : np.sum((resF/(1-mProp) - target) ** 2),
'err_norm' : np.sum((resF/(1-par[3]) - target) ** 2)/np.sum(target ** 2),
'cost' : cost 
}

In [ ]:
u0 = np.concatenate((mProp * dinit, (1 - mProp) * dinit))
u0M, u0F = sushibelt.aggregate_segments(u0, segIdx, expD['Abbreviation'], fun=np.sum)
(np.sum(dinit),np.sum(u0M),np.sum(u0F),u0M,u0F,np.sum(u0[:n]),np.sum(u0[n:]))

In [ ]:
x=np.array(range(expD.shape[0]))

plt.plot(x,nobsDnn,'o-',label=cname7)
#plt.plot(x,nresM,label='mobile O')
#plt.plot(x,nresF,label='detached O')
plt.plot(x,resM,label='mobile DC')
plt.plot(x,resF,label='detached DC')
# show a legend on the plot
plt.legend()
plt.title(f"Cost = {cost}") 
# function to show the plot
#plt.show()
plt.savefig('bestFitEdita20240530.png')
plt.savefig('bestFitEdita20240530.pdf')
plt.savefig('bestFitEdita20240530.svg')


In [ ]:
x=np.array(range(expD.shape[0]))
fig, ax = plt.subplots()
ax.plot(x,nobsDnn,'o-',label=cname7)
ax.plot(x,resF/(1-mProp),label='detached norm DC')
# show a legend on the plot
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
#plt.title(f"Cost = {cost}") 
plt.title(f"Cost = {format(cost,'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
#plt.show()
plt.savefig('bestFitEdita20240530.fit.png')
plt.savefig('bestFitEdita20240530.fit.pdf')
plt.savefig('bestFitEdita20240530.fit.svg')


Day 7 3 weeks
----

In [ ]:
cname='D7W3'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"case1_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
utarg /= np.sum(utarg)

# Run simulation
initTime=time.time()
A = get_sys_matrix(utarg,F=0,Ctau=1e-5,dv=1e-9)
utrace = run_sim(A)
FinalTime=time.time()-initTime

# snapshot of steady-state
snap_ss(f"case1_{cname}")

# animation
#animate_sim()
print(FinalTime)

Day 0 3 month
----

In [ ]:
cname='D0M3'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"case1_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
utarg /= np.sum(utarg)

# Run simulation
initTime=time.time()
A = get_sys_matrix(utarg,F=0,Ctau=1e-5,dv=1e-9)
utrace = run_sim(A)
FinalTime=time.time()-initTime

# snapshot of steady-state
snap_ss(f"case1_{cname}")

# animation
#animate_sim()
print(FinalTime)

Day 7 3 month
----

In [ ]:
cname='D7M3'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"case1_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
utarg /= np.sum(utarg)

# Run simulation
initTime=time.time()
A = get_sys_matrix(utarg,F=0,Ctau=1e-5,dv=1e-9)
utrace = run_sim(A)
FinalTime=time.time()-initTime

# snapshot of steady-state
snap_ss(f"case1_{cname}")

# animation
#animate_sim()
print(FinalTime)

Day 0 18 month
----

In [ ]:
cname='D0M18'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"case1_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
utarg /= np.sum(utarg)

# Run simulation
initTime=time.time()
A = get_sys_matrix(utarg,F=0,Ctau=1e-5,dv=1e-9)
utrace = run_sim(A)
FinalTime=time.time()-initTime

# snapshot of steady-state
snap_ss(f"case1_{cname}")

# animation
#animate_sim()
print(FinalTime)

Day 7 18 month
----

In [ ]:
cname='D7M18'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"case1_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
utarg /= np.sum(utarg)

# Run simulation
initTime=time.time()
A = get_sys_matrix(utarg,F=0,Ctau=1e-5,dv=1e-9)
utrace = run_sim(A)
FinalTime=time.time()-initTime

# snapshot of steady-state
snap_ss(f"case1_{cname}")

# animation
#animate_sim()
print(FinalTime)

Second Case: timescales not separated, proximal inputs are *greedy* and prevent distal transport
----

Day 0 3 weeks
----

In [ ]:
cname='D0W3'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"case2_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
# Run simulation
utarg /= np.sum(utarg)
initTime=time.time()
A = get_sys_matrix(utarg,F=1,Ctau=1e-4,dv=1e-9)
utrace = run_sim(A)
FinalTime=time.time()-initTime

# snapshot of steady-state
snap_ss(f"case2_{cname}")

# animation
#animate_sim()
print(FinalTime)

Day 7 3 weeks
----

In [ ]:
cname='D7W3'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"case2_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
utarg /= np.sum(utarg)

# Run simulation
initTime=time.time()
A = get_sys_matrix(utarg,F=1,Ctau=1e-4,dv=1e-9)
utrace = run_sim(A)
FinalTime=time.time()-initTime

# snapshot of steady-state
snap_ss(f"case2_{cname}")

# animation
#animate_sim()
print(FinalTime)

Day 0 3 month
----

In [ ]:
cname='D0M3'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"case2_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
utarg /= np.sum(utarg)

# Run simulation
initTime=time.time()
A = get_sys_matrix(utarg,F=1,Ctau=1e-4,dv=1e-9)
utrace = run_sim(A)
FinalTime=time.time()-initTime

# snapshot of steady-state
snap_ss(f"case2_{cname}")

# animation
#animate_sim()
print(FinalTime)

Day 7 3 month
----

In [ ]:
cname='D7M3'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"case2_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
utarg /= np.sum(utarg)

# Run simulation
initTime=time.time()
A = get_sys_matrix(utarg,F=1,Ctau=1e-4,dv=1e-9)
utrace = run_sim(A)
FinalTime=time.time()-initTime

# snapshot of steady-state
snap_ss(f"case2_{cname}")

# animation
#animate_sim()
print(FinalTime)

Day 0 18 month
----

In [ ]:
cname='D0M18'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"case2_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
utarg /= np.sum(utarg)

# Run simulation
initTime=time.time()
A = get_sys_matrix(utarg,F=1,Ctau=1e-4,dv=1e-9)
utrace = run_sim(A)
FinalTime=time.time()-initTime

# snapshot of steady-state
snap_ss(f"case2_{cname}")

# animation
#animate_sim()
print(FinalTime)

Day 7 18 month
----

In [ ]:
cname='D7M18'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"case2_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
utarg /= np.sum(utarg)

# Run simulation
initTime=time.time()
A = get_sys_matrix(utarg,F=1,Ctau=1e-4,dv=1e-9)
utrace = run_sim(A)
FinalTime=time.time()-initTime

# snapshot of steady-state
snap_ss(f"case2_{cname}")

# animation
#animate_sim()
print(FinalTime)

Third Case: Time scales separated, different trafficking strategy
------

Day 0 3 weeks
----

In [ ]:
cname='D0W3'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"case3_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
# Run simulation
utarg /= np.sum(utarg)
initTime=time.time()
A = get_sys_matrix(utarg,F=0,Ctau=2e-4,dv=1e-9)
utrace = run_sim(A)
FinalTime=time.time()-initTime

# snapshot of steady-state
snap_ss(f"case3_{cname}")

# animation
#animate_sim()
print(FinalTime)

Day 7 3 weeks
----

In [ ]:
cname='D7W3'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"case3_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
utarg /= np.sum(utarg)

# Run simulation
initTime=time.time()
A = get_sys_matrix(utarg,F=0,Ctau=2e-4,dv=1e-9)
utrace = run_sim(A)
FinalTime=time.time()-initTime

# snapshot of steady-state
snap_ss(f"case3_{cname}")

# animation
#animate_sim()
print(FinalTime)

Day 0 3 month
----

In [ ]:
cname='D0M3'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"case3_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
utarg /= np.sum(utarg)

# Run simulation
initTime=time.time()
A = get_sys_matrix(utarg,F=0,Ctau=2e-4,dv=1e-9)
utrace = run_sim(A)
FinalTime=time.time()-initTime

# snapshot of steady-state
snap_ss(f"case3_{cname}")

# animation
#animate_sim()
print(FinalTime)

Day 7 3 month
----

In [ ]:
cname='D7M3'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"case3_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
utarg /= np.sum(utarg)

# Run simulation
initTime=time.time()
A = get_sys_matrix(utarg,F=0,Ctau=2e-4,dv=1e-9)
utrace = run_sim(A)
FinalTime=time.time()-initTime

# snapshot of steady-state
snap_ss(f"case3_{cname}")

# animation
#animate_sim()
print(FinalTime)

Day 0 18 month
----

In [ ]:
cname='D0M18'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"case3_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
utarg /= np.sum(utarg)

# Run simulation
initTime=time.time()
A = get_sys_matrix(utarg,F=0,Ctau=2e-4,dv=1e-9)
utrace = run_sim(A)
FinalTime=time.time()-initTime

# snapshot of steady-state
snap_ss(f"case3_{cname}")

# animation
#animate_sim()
print(FinalTime)

Day 7 18 month
----

In [ ]:
cname='D7M18'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"case3_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
utarg /= np.sum(utarg)

# Run simulation
initTime=time.time()
A = get_sys_matrix(utarg,F=0,Ctau=2e-4,dv=1e-9)
utrace = run_sim(A)
FinalTime=time.time()-initTime

# snapshot of steady-state
snap_ss(f"case3_{cname}")

# animation
#animate_sim()
print(FinalTime)

In [ ]:
utrace[743:753,10]

Third Case: protein + RNA
----

In [ ]:
initTime=time.time()
A,u,t,excess,err = sushibelt.run_uniform_reattachment(h, 0.1, 5, 50.0, 1e-5,1e-6)
FinalTime=time.time()-initTime
print(FinalTime)
#sushibelt.snapshots(h,u,t,'plots/with_reattach',cellname,view,u_cmap,us_cmap)
#print 'excess cargo = %2.1f percent'%(excess[-1])

In [ ]:
N
utrace=u[:,:2*N]
utrace=utrace.T

In [ ]:
A.shape

In [ ]:
t.shape

In [ ]:
u.shape

In [ ]:
nframes=50
t = np.logspace(-0.5,8.5,nframes)
t.shape

In [ ]:
u0 = np.zeros(4*n)
u0[0] = 1.0  # compartment 0 is the soma
initTime=time.time()
utrace = [u0]
w,V = scipy.linalg.eig(A)
Vinv = np.linalg.inv(V)
t = np.logspace(-0.5,8.5,nframes)
for t_ in t: utrace.append(solve_u(w,V,Vinv,t_))

FinalTime=time.time()-initTime
print(FinalTime)


cvals=utrace[:,-1]
cvals.shape

In [ ]:
(1484/len(seg_list),len(seg_list))

In [ ]:
t.shape

In [ ]:
[d/4 for d in A.shape]

In [ ]:
(1484*2,A.shape)

In [ ]:
max(dts)

In [ ]:
from PyNeuronToolbox.morphology import shapeplot, allsec_preorder, root_indices, shapeplot_animate
roots = root_indices(allsec_preorder(h))
roots

In [ ]:
import pandas as pd
fitdt = pd.read_csv('data/Distr.csv')

In [ ]:
idx = []
for i in range(n):
    if abs(dts[i]-100)<1 :
        idx.append(1)
    else:
        idx.append(0)
sum(idx)

In [ ]:
[ (i,j) for i in range(fitdt.shape[0]) for j in range(n) if abs(dts[j]-fitdt['X'][i])<1 ]

In [ ]:
type(fitdt)

In [ ]:
seg_idx = [(i, j) for i in range(fitdt.shape[0]) for j in range(n) if abs(dts[j] - fitdt['X'][i]) < 1]
len(seg_idx)

In [ ]:
inum=3
ix=[ i for i in range(3*inum)]
(ix[:inum],ix[inum:2*inum],ix[2*inum:])

In [ ]:
[ max(u[:,r]) for r in range(u.shape[0])]

In [ ]:
range(u.shape[1])

In [ ]:
    clim=[0,0.00022]
    uplt = u[:,n:2*n]
    uplt = uplt.T
    fig = plt.figure(figsize=(8,8))
    shapeax = plt.subplot(111, projection='3d')
    lines = shapeplot(h,shapeax,cvals=uplt[:,1],clim=clim,\
                      cmap=plt.cm.cool,order='pre',lw=2)
    shapeax.view_init(elev=80,azim=-90)
    plt.title('free RNA 1',fontweight='bold',fontsize=14)

    # colorbar
    sm = cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm = plt.cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm._A = []
    #plt.colorbar(sm, shrink=0.5)
    #plt.colorbar(sm,cax=shapeax)
    #plt.colorbar()
    plt.tight_layout()
    plt.axis('off')


In [ ]:
    clim=[0,0.00022]
    uplt = u[:,:n]
    uplt = uplt.T
    fig = plt.figure(figsize=(8,8))
    shapeax = plt.subplot(111, projection='3d')
    lines = shapeplot(h,shapeax,cvals=uplt[:,-1],clim=clim,\
                      cmap=plt.cm.cool,order='pre',lw=2)
    shapeax.view_init(elev=80,azim=-90)
    plt.title('bounded RNA -1',fontweight='bold',fontsize=14)

    # colorbar
    sm = cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm = plt.cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm._A = []
    #plt.colorbar(sm, shrink=0.5)
    #plt.colorbar(sm,cax=shapeax)
    #plt.colorbar()
    plt.tight_layout()
    plt.axis('off')


In [ ]:
    clim=[0,0.00022]
    uplt = u[:,2*n:3*n]
    uplt = uplt.T
    fig = plt.figure(figsize=(8,8))
    shapeax = plt.subplot(111, projection='3d')
    lines = shapeplot(h,shapeax,cvals=uplt[:,-1],clim=clim,\
                      cmap=plt.cm.cool,order='pre',lw=2)
    shapeax.view_init(elev=80,azim=-90)
    plt.title('bound protein -1',fontweight='bold',fontsize=14)

    # colorbar
    sm = cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm = plt.cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm._A = []
    #plt.colorbar(sm, shrink=0.5)
    #plt.colorbar(sm,cax=shapeax)
    #plt.colorbar()
    plt.tight_layout()
    plt.axis('off')


In [ ]:
    clim=[0,0.00022]
    uplt = u[:,3*n:]
    uplt = uplt.T
    fig = plt.figure(figsize=(8,8))
    shapeax = plt.subplot(111, projection='3d')
    lines = shapeplot(h,shapeax,cvals=uplt[:,-1],clim=clim,\
                      cmap=plt.cm.cool,order='pre',lw=2)
    shapeax.view_init(elev=80,azim=-90)
    plt.title('free protein -1',fontweight='bold',fontsize=14)

    # colorbar
    sm = cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm = plt.cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm._A = []
    #plt.colorbar(sm, shrink=0.5)
    #plt.colorbar(sm,cax=shapeax)
    #plt.colorbar()
    plt.tight_layout()
    plt.axis('off')


In [ ]:
u0 = np.zeros(2*N+1)
n = int(u0.shape[0]/2)
(u0.shape,n,N)

In [ ]:
bgSignal = 1e-5
par0 = [0.8861278052764772,-5.358123919558814,0.3127455791698551,-5.7356821635860165,-17.999032750284783,0.21382364663804718,0.06074280874523973,0.04781382417616123,0.04562340830020193,1.0000000000039887e-07,0.7733439929081787,0.5543045057407645,0.1468913931288651,0.14710118884157364,0.1316203146580693,0.15499357159791044,0.17296720163305693,0.20287137824434023,0.3735878952746845,0.5243019440900127,0.043864102155039865,0.1966480252602025,0.19591970216660096,0.2755074889018492,0.9999999999999992]
cost0 = 3.331839554992909e-11
sec_list = allsec_preorder(h)
seg_list = []
for sec in sec_list:
    locs = np.linspace(0,1,sec.nseg+2)[1:-1]
    for loc in locs:
        seg_list.append(sec(loc))
N = len(seg_list)
tdf=pd.read_csv('data/seg_mapping.csv')
abbCA1=tdf['abb']
abbT={}
segIdx={}
for i in range(N):
    abbT[abbCA1[i]] = 1+ abbT.get(abbCA1[i],0)
    ll=segIdx.get(abbCA1[i],[])
    ll.append(i)
    segIdx[abbCA1[i]] = ll

expD=pd.read_csv('data/CA1_gradient.csv')
subreg = ['CA1so', 'CA1sr', 'CA1slm']

#cname='D0W3'
cname0='D0M3'
d0w = -1 * np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d0w[sidx] *= -1*expD[f"{cname0}_MEAN"][i]/len(sidx)
for i in range(N):
    if d0w[i]<0:
        d0w[i] = bgSignal

#cname='D7W3'
cname7='D7M3'
d7w = -1*np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d7w[sidx] *= -1 * expD[f"{cname7}_MEAN"][i]/len(sidx)
for i in range(N):
    if d7w[i]<0:
        d7w[i] = bgSignal
day7 = 7 * 24 * 3600 # final time point

itarg = np.ones(N, dtype=int)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx = segIdx[abb]
    itarg[sidx] *= (i+5)


In [ ]:
dinit = d0w/np.sum(d0w)
target = (np.array(expD[f"{cname0}_MEAN"])-np.array(expD[f"{cname7}_MEAN"]))/np.sum(expD[f"{cname0}_MEAN"]) #norm target to Day0 sum to take into accound degradation
targSD = (np.array(expD[f"{cname0}_SD"])+np.array(expD[f"{cname7}_SD"]))/np.sum(expD[f"{cname0}_MEAN"]) #results to fit to
(target,targSD,expD[f"{cname0}_SD"],expD[f"{cname7}_SD"])

In [ ]:
expD

In [ ]:
(len(par0)*5*50*5)/3600

In [ ]:
resp = np.zeros((fitdt.shape[0],u.shape[0]))
resp.shape

In [ ]:
resp = np.zeros((fitdt.shape[0],u.shape[0]))
for i in range(seg_idx.shape[0]):
    ii = int(seg_idx[i,0])
    ij = int(seg_idx[i,1])
    resp[ii] += u[:,ij].T
resp

In [ ]:
dist = (resp-fitdt['V']) ** 2
dist

In [ ]:
fitdt['V']

In [ ]:
[ sum((resp[:,i] - fitdt['V']) ** 2) for i in range(resp.shape[1])]